In [ ]:
# === CÉLULA 0 — Definições base: MazeBlocks, gen e plot_maze ===
from enum import IntEnum
import random
import matplotlib.pyplot as plt

class MazeBlocks(IntEnum):
    WALL = 0
    PASSAGE = 1

def gen(size: int) -> list[list[int]]:
    """
    Gera um labirinto perfeito (DFS backtracking) numa grade física (2*size+1)x(2*size+1).
    Retorna 0=WALL, 1=PASSAGE. Abre entrada em (0,1) e saída em (phys-1, phys-2).
    """
    if size < 1:
        raise ValueError("size must be >= 1")

    phys = size * 2 + 1
    maze_map: list[list[int]] = [[MazeBlocks.WALL for _ in range(phys)] for _ in range(phys)]

    def get_neighbors(r: int, c: int) -> list[tuple[int, int]]:
        out: list[tuple[int, int]] = []
        for dx, dy in [(-2,0), (2,0), (0,-2), (0,2)]:
            x, y = r + dx, c + dy
            if 0 <= x < phys and 0 <= y < phys and maze_map[x][y] != MazeBlocks.PASSAGE:
                out.append((x, y))
        return out

    stack: list[tuple[int, int]] = [(1, 1)]
    maze_map[1][1] = MazeBlocks.PASSAGE

    while stack:
        i, j = stack[-1]
        neighbors = get_neighbors(i, j)
        if neighbors:
            ni, nj = random.choice(neighbors)
            maze_map[(i + ni)//2][(j + nj)//2] = MazeBlocks.PASSAGE  # “abre” a parede entre os dois
            maze_map[ni][nj] = MazeBlocks.PASSAGE
            stack.append((ni, nj))
        else:
            stack.pop()

    # Entrada e saída nas bordas
    maze_map[0][1] = MazeBlocks.PASSAGE
    maze_map[phys - 1][phys - 2] = MazeBlocks.PASSAGE

    # Converte IntEnum -> int
    return [[int(cell) for cell in row] for row in maze_map]

def plot_maze(grid: list[list[int]], title: str | None = None) -> None:
    plt.figure(figsize=(6, 6))
    plt.imshow(grid, cmap='gray', interpolation='nearest')
    if title: plt.title(title)
    plt.axis('off'); plt.gca().invert_yaxis()
    plt.show()


In [ ]:
# === CÉLULA 1 — Labirinto com múltiplos caminhos (braided) ===
import random
import matplotlib.pyplot as plt

# ---- suas defs já existentes (devem estar acima) ----
# from enum import IntEnum
# class MazeBlocks(IntEnum): ...
# def gen(size: int) -> list[list[int]]: ...
# def plot_maze(grid, title=None): ...

# ---------------- utilitários ----------------
def build_graph_from_maze(grid):
    h, w = len(grid), len(grid[0])
    g = {}
    for r in range(h):
        for c in range(w):
            if grid[r][c] == 1:
                nbs = []
                for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                    rr, cc = r+dr, c+dc
                    if 0 <= rr < h and 0 <= cc < w and grid[rr][cc] == 1:
                        nbs.append((rr, cc))
                g[(r,c)] = nbs
    return g

def find_entrance_exit(grid):
    h, w = len(grid), len(grid[0])
    start = next(((0,c) for c in range(w) if grid[0][c] == 1), None)
    goal  = next(((h-1,c) for c in range(w-1, -1, -1) if grid[h-1][c] == 1), None)
    if start is None or goal is None:
        raise RuntimeError("Não encontrei entrada/saída nas bordas.")
    return start, goal

def graph_stats(g):
    """Retorna (n_nodes, n_edges_undirected, n_components, loops_est=E-V+C)."""
    V = len(g)
    E2 = sum(len(nbs) for nbs in g.values())  # soma graus (duas vezes as arestas)
    E = E2 // 2
    seen = set()
    comps = 0
    for v in g:
        if v not in seen:
            comps += 1
            stack = [v]; seen.add(v)
            while stack:
                u = stack.pop()
                for nb in g[u]:
                    if nb not in seen:
                        seen.add(nb); stack.append(nb)
    loops = E - V + comps
    return V, E, comps, loops

# ------------- "trançar" o labirinto (criar ciclos) -------------
def braid_maze(grid, p: float = 0.20, seed=None):
    """
    Abre paredes entre passagens com probabilidade p para criar ciclos.
    Mantém a topologia do gerador (passagens em índices ímpares).
    - p=0.0: labirinto perfeito (1 caminho único)
    - p≈0.05–0.30: alguns loops
    - p>0.35: bem trançado
    """
    rng = random.Random(seed)
    h, w = len(grid), len(grid[0])
    for r in range(1, h-1):
        for c in range(1, w-1):
            if grid[r][c] != 0:
                continue
            # parede vertical entre duas passagens
            if (r % 2 == 0) and (c % 2 == 1) and grid[r-1][c] == 1 and grid[r+1][c] == 1:
                if rng.random() < p:
                    grid[r][c] = 1
            # parede horizontal entre duas passagens
            if (r % 2 == 1) and (c % 2 == 0) and grid[r][c-1] == 1 and grid[r][c+1] == 1:
                if rng.random() < p:
                    grid[r][c] = 1
    return grid

# ---- escolha do experimento ----
SIZE = 22
SEED = None        # ou um inteiro (ex.: 42) para reprodutibilidade
P_BRAID = 0.05     # 0.0 = perfeito; >0 cria múltiplos caminhos

# ---- gerar e, opcionalmente, "trançar" ----
random.seed(SEED)
grid = gen(SIZE)
if P_BRAID and P_BRAID > 0:
    braid_maze(grid, p=P_BRAID, seed=SEED)

start, goal = find_entrance_exit(grid)
g = build_graph_from_maze(grid)

# stats para confirmar que há ciclos
V, E, C, L = graph_stats(g)  # L ≈ número de loops independentes
status = "perfect (1 caminho)" if L == 0 else f"braided (loops≈{L})"
print(f"[maze] size={SIZE}, seed={SEED}, phys={2*SIZE+1}, edges={E}, nodes={V}, comps={C}, {status}")

# guarda no CTX
CTX = {
    "size": SIZE,
    "seed": SEED,
    "p_braid": P_BRAID,
    "grid": grid,
    "start": start,
    "goal": goal,
    "graph": g,
    "loops_est": L,
}

plot_maze(grid, f"Maze (size={SIZE}, p={P_BRAID}, {status})")
plt.show()


In [ ]:
from heapq import heappush, heappop
import matplotlib.pyplot as plt

def astar(g, start, goal):
    def h(a, b):  # Manhattan
        return abs(a[0]-b[0]) + abs(a[1]-b[1])

    open_set = [(h(start, goal), start)]
    came_from = {}
    cost = {start: 0}
    visited = []  # ordem de expansão (para sabermos “nós visitados”)

    def rebuild(n):
        p = [n]
        while n in came_from:
            n = came_from[n]
            p.append(n)
        return list(reversed(p))

    while open_set:
        f_curr, curr = heappop(open_set)
        # guarda contra entradas obsoletas (opcional)
        if f_curr > cost.get(curr, float("inf")) + h(curr, goal):
            continue

        visited.append(curr)

        if curr == goal:
            return {
                "path": rebuild(curr),
                "came_from": came_from,
                "visited": visited,
            }

        for nb in g.get(curr, ()):
            new_cost = cost[curr] + 1
            if new_cost < cost.get(nb, float("inf")):
                cost[nb] = new_cost
                heappush(open_set, (new_cost + h(nb, goal), nb))
                came_from[nb] = curr
    return {"path": None, "came_from": {}, "visited": visited}

def plot_nodes_and_edges(grid, path, visited, came_from, title=None):
    """
    Desenha:
      - caminho final (arestas e nós) em vermelho
      - nós visitados fora do caminho em verde
      - (opcional) também desenha arestas “visitadas” fora do caminho (ver flag abaixo)
    """
    show_nonpath_edges = True  # deixe True para também ligar com linha verde os nós visitados ao seu pai

    plt.figure(figsize=(6,6))
    plt.imshow(grid, cmap='gray', interpolation='nearest')
    ax = plt.gca()
    ax.invert_yaxis()
    ax.axis('off')
    if title:
        ax.set_title(title)

    path = path or []
    path_set = set(path)

    # 1) nós visitados que NÃO estão no caminho final (verdes)
    nonpath_nodes = [n for n in visited if n not in path_set]
    if nonpath_nodes:
        xs = [c for r,c in nonpath_nodes]
        ys = [r for r,c in nonpath_nodes]
        plt.scatter(xs, ys, s=12, marker='o', color='green')

    # 2) arestas “visitadas” fora do caminho (verde)
    if show_nonpath_edges:
        for n in nonpath_nodes:
            p = came_from.get(n)
            if p is not None and p not in path_set:
                x = [p[1], n[1]]
                y = [p[0], n[0]]
                plt.plot(x, y, linewidth=1, color='green')

    # 3) caminho final — arestas (vermelho)
    for i in range(len(path)-1):
        a, b = path[i], path[i+1]
        plt.plot([a[1], b[1]], [a[0], b[0]], linewidth=2, color='red')

    # 4) caminho final — nós (vermelho)
    if path:
        xs = [c for r,c in path]
        ys = [r for r,c in path]
        plt.scatter(xs, ys, s=14, marker='o', color='red')

    # 5) start/goal marcados
    if path:
        s, t = path[0], path[-1]
    else:
        s, t = CTX["start"], CTX["goal"]
    plt.scatter([s[1]], [s[0]], s=40, marker='s')  # cor padrão
    plt.scatter([t[1]], [t[0]], s=40, marker='X')

    plt.show()

# ---- usa o mesmo labirinto da célula 1 ----
grid = CTX["grid"]
g     = CTX["graph"]
start = CTX["start"]
goal  = CTX["goal"]
size  = CTX["size"]
seed  = CTX["seed"]

res_astar = astar(g, start, goal)
plot_nodes_and_edges(grid, res_astar["path"], res_astar["visited"], res_astar["came_from"],
                     f"A* (size={size}, seed={seed})")

# opcional: guardar no CTX pra reutilizar na célula 4
CTX["astar"] = res_astar


In [ ]:
from heapq import heappush, heappop
import matplotlib.pyplot as plt

def greedy_best_first(g, start, goal):
    def h(a, b):  # Manhattan
        return abs(a[0]-b[0]) + abs(a[1]-b[1])

    open_set = [(h(start, goal), start)]
    came_from = {}
    visited = []
    seen = set([start])

    def rebuild(n):
        p = [n]
        while n in came_from:
            n = came_from[n]
            p.append(n)
        return list(reversed(p))

    while open_set:
        _, curr = heappop(open_set)
        visited.append(curr)
        if curr == goal:
            return {"path": rebuild(curr), "came_from": came_from, "visited": visited}
        for nb in g.get(curr, ()):
            if nb in seen:
                continue
            seen.add(nb)
            came_from[nb] = curr
            heappush(open_set, (h(nb, goal), nb))
    return {"path": None, "came_from": {}, "visited": visited}

# (reusa a plot_nodes_and_edges da célula 2)

# ---- usa o mesmo labirinto da célula 1 ----
grid = CTX["grid"]
g     = CTX["graph"]
start = CTX["start"]
goal  = CTX["goal"]
size  = CTX["size"]
seed  = CTX["seed"]

res_greedy = greedy_best_first(g, start, goal)
plot_nodes_and_edges(grid, res_greedy["path"], res_greedy["visited"], res_greedy["came_from"],
                     f"Greedy Best-First (size={size}, seed={seed})")

# opcional: guardar no CTX pra reutilizar na célula 4
CTX["greedy"] = res_greedy


In [ ]:
# === CÉLULA 4 — COMPARAÇÃO VISUAL A* x GULOSA COM NÓS E ARESTAS ===
import matplotlib.pyplot as plt

def draw_nodes_and_edges(ax, grid, path, visited, came_from, title=None,
                         show_nonpath_edges=True):
    ax.imshow(grid, cmap='gray', interpolation='nearest')
    ax.invert_yaxis(); ax.axis('off')
    if title: ax.set_title(title)

    path = path or []
    path_set = set(path)

    # verdes: visitados fora do caminho
    nonpath_nodes = [n for n in visited if n not in path_set]
    if nonpath_nodes:
        xs = [c for r,c in nonpath_nodes]
        ys = [r for r,c in nonpath_nodes]
        ax.scatter(xs, ys, s=12, marker='o', color='green')

    if show_nonpath_edges:
        for n in nonpath_nodes:
            p = came_from.get(n)
            if p is not None and p not in path_set:
                ax.plot([p[1], n[1]], [p[0], n[0]], linewidth=1, color='green')

    # vermelho: caminho final (arestas + nós)
    for i in range(len(path)-1):
        a, b = path[i], path[i+1]
        ax.plot([a[1], b[1]], [a[0], b[0]], linewidth=2, color='red')
    if path:
        xs = [c for r,c in path]
        ys = [r for r,c in path]
        ax.scatter(xs, ys, s=14, marker='o', color='red')

    # start/goal
    s = CTX["start"]; t = CTX["goal"]
    ax.scatter([s[1]], [s[0]], s=40, marker='s')
    ax.scatter([t[1]], [t[0]], s=40, marker='X')

# ---- pega do CTX (se não tiver, roda células 2 e 3 antes) ----
grid = CTX["grid"]
size = CTX["size"]; seed = CTX["seed"]

res_astar  = CTX.get("astar",  None)
res_greedy = CTX.get("greedy", None)
assert res_astar is not None and res_greedy is not None, "Rode as células 2 e 3 primeiro."

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
draw_nodes_and_edges(axes[0], grid, res_astar["path"],  res_astar["visited"],  res_astar["came_from"],
                     title="A* — caminho (vermelho) • visitados fora do caminho (verde)")
draw_nodes_and_edges(axes[1], grid, res_greedy["path"], res_greedy["visited"], res_greedy["came_from"],
                     title="Gulosa — caminho (vermelho) • visitados fora do caminho (verde)")
plt.tight_layout()
plt.show()


In [ ]:
# === CÉLULA 5 — ANIMAÇÃO DA BUSCA (nós/arestas) ===
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.collections import LineCollection
from IPython.display import HTML

def animate_search_nodes_edges(grid,
                               visited,
                               came_from,
                               final_path=None,
                               title="Animação da busca",
                               interval_ms=20,
                               show_green_edges=True,
                               green_point_size=10,
                               red_point_size=12):
    """
    Anima:
      • expansão (visited) em VERDE: pontos e (opcional) arestas para o pai;
      • no último frame, CAMINHO FINAL em VERMELHO (arestas + nós).
    """
    assert isinstance(visited, (list, tuple)) and len(visited) > 0, "Lista 'visited' está vazia."

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(grid, cmap='gray', interpolation='nearest')
    ax.axis('off')
    ax.invert_yaxis()
    ax.set_title(title)

    # start/goal
    s = CTX["start"]; t = CTX["goal"]
    ax.scatter([s[1]], [s[0]], s=40, marker='s')
    ax.scatter([t[1]], [t[0]], s=40, marker='X')

    # artistas
    green_xs, green_ys = [], []
    green_scatter = ax.scatter([], [], s=green_point_size, marker='o', color='green')
    green_segments = LineCollection([], colors='green', linewidths=1)
    ax.add_collection(green_segments)

    red_path_line, = ax.plot([], [], linewidth=2, color='red')
    red_scatter = ax.scatter([], [], s=red_point_size, color='red')

    final_path = final_path or []

    def init():
        empty = np.empty((0, 2))
        green_scatter.set_offsets(empty)
        green_segments.set_segments([])
        red_path_line.set_data([], [])
        red_scatter.set_offsets(empty)
        return (green_scatter, green_segments, red_path_line, red_scatter)

    def update(frame):
        node = visited[frame]
        r, c = node

        # ponto verde
        green_xs.append(c)
        green_ys.append(r)
        green_scatter.set_offsets(np.column_stack([green_xs, green_ys]))

        # aresta verde p/ o pai (se houver)
        if show_green_edges:
            parent = came_from.get(node)
            if parent is not None:
                pr, pc = parent
                segs = list(green_segments.get_segments())
                segs.append([(pc, pr), (c, r)])
                green_segments.set_segments(segs)

        # último frame: caminho final
        if frame == len(visited) - 1 and final_path:
            px = [c for rr, c in final_path]
            py = [rr for rr, c in final_path]
            red_path_line.set_data(px, py)
            red_scatter.set_offsets(np.column_stack([px, py]))

        return (green_scatter, green_segments, red_path_line, red_scatter)

    anim = animation.FuncAnimation(
        fig, update, init_func=init,
        frames=len(visited), interval=interval_ms, blit=True
    )
    plt.close(fig)
    return anim

# ---- recuperar resultados do CTX (gerados nas células 2 e 3) ----
assert "astar" in CTX and "greedy" in CTX, "Rode as células 2 e 3 antes da animação."
grid = CTX["grid"]
res_astar  = CTX["astar"]
res_greedy = CTX["greedy"]

# ---- criar animações ----
anim_astar  = animate_search_nodes_edges(
    grid,
    visited=res_astar["visited"],
    came_from=res_astar["came_from"],
    final_path=res_astar["path"],
    title="A* — expansão (verde) + caminho final (vermelho)",
    interval_ms=15,
)

anim_greedy = animate_search_nodes_edges(
    grid,
    visited=res_greedy["visited"],
    came_from=res_greedy["came_from"],
    final_path=res_greedy["path"],
    title="Gulosa — expansão (verde) + caminho final (vermelho)",
    interval_ms=15,
)

# ---- exibir no Jupyter ----
display(HTML(anim_astar.to_jshtml()))
display(HTML(anim_greedy.to_jshtml()))
